# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Lee  ->  K. Lee  |  ['K. Lee']
C. Fendt  ->  C. Fendt  |  ['C. Fendt']
D. Hogg  ->  D. Hogg  |  ['D. Hogg']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
Arxiv has 82 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.02801


extracting tarball to tmp_2510.02801...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.03037
extracting tarball to tmp_2510.03037...

 done.


C. Fendt  ->  C. Fendt  |  ['C. Fendt']


Found 126 bibliographic references in tmp_2510.03037/ms_giri.bbl.
Retrieving document from  https://arxiv.org/e-print/2510.03221


extracting tarball to tmp_2510.03221... done.


H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Found 102 bibliographic references in tmp_2510.03221/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.03037-b31b1b.svg)](https://arxiv.org/abs/2510.03037) | **Probing the formation of megaparsec-scale giant radio galaxies II. Continuum & polarization behavior from MHD simulations**  |
|| G. Giri, et al. -- incl., <mark>C. Fendt</mark> |
|*Appeared on*| *2025-10-06*|
|*Comments*| *Accepted for publication in A&A journal*|
|**Abstract**|            The persistence of radiative signatures in giant radio galaxies remains a frontier topic of research, with contemporary telescopes revealing intricate features that require investigation. This study aims to examine the emission characteristics of simulated GRGs, and correlate them with their underlying 3D dynamical properties. Sky-projected continuum and polarization maps at 1 GHz were computed from five 3D-RMHD simulations by integrating the synthesized emissivity data along the line of sight, with the integration path chosen to reflect the GRG evolution in the sky plane. The emissivities were derived from these RMHD simulations, featuring FR-I and FR-II jets injected from different locations of the large-scale environment. The jet-cocoon morphologies are strongly shaped by the triaxiality of the environment, resulting in features like wings and asymmetric cocoons, thereby making morphology a crucial indicator of GRG formation mechanisms. The decollimation of the bulk flow in GRG jets gives rise to intricate cocoon features, most notably filamentary structures-magnetically dominated threads with lifespans of a few Myr. High-jet-power cases frequently display enhanced emission zones at mid-cocoon distances (alongside warmspots around the jet-head), contradicting the interpretations of the GRG as a restarting source. In such cases, examining the lateral intensity variation of the cocoon may reveal the source's state, with a gradual decrease in emission suggesting a low-active stage. This study highlights that applying a simple radio power-jet power relation to a statistical GRG sample is unfeasible, as it depends on growth conditions of individual GRGs. Effects such as inverse-Compton CMB cooling and matter entrainment significantly impact the long-term emission persistence of GRGs. The diminishing fractional polarization with GRG evolution reflects increasing turbulence in the cocoon.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.03221-b31b1b.svg)](https://arxiv.org/abs/2510.03221) | **Inferring Stellar Densities with Flexible Models I: The Distribution of RR Lyrae in the Milky Way with $\textit{Gaia}$ DR3**  |
|| M. Lucey, et al. -- incl., <mark>D. Hogg</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-10-06*|
|*Comments*| *13 pages, 6 figures, submitted to ApJ, comments welcome!*|
|**Abstract**|            Understanding the formation and evolutionary history of the Milky Way requires detailed mapping of its stellar components, which preserve fossil records of the Galaxy's assembly through cosmic time. RR Lyrae stars are particularly well-suited for this endeavor, as they are old, standard candle variables that probe the Galaxy's earliest formation epochs. In this work, we employ a hierarchical Bayesian Gaussian Mixture Model (GMM) to characterize the three-dimensional density distribution of RR Lyrae stars in the Milky Way. This approach provides a flexible framework for modeling complex stellar distributions, particularly in the inner Galaxy where the bulge, disk, and halo components overlap. Our analysis reveals that the inner Galaxy is dominated by a distinct prolate stellar population with axis ratio $q$=1.30. Consistent with previous work, we find the halo follows a $r^{-4}$ power-law profile that flattens within 12 kpc of the Galactic center. We also confirm the halo is oblate ($q$=0.62) with a tilt angle of $12.22^{\circ}$. We report for the first time that this tilt aligns the halo major axis in the direction of the Sagittarius dwarf galaxy. These results establish GMMs as an effective and flexible tool for modeling Galactic structure and provide new constraints on the distribution of old stars in the inner Galaxy.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.02801-b31b1b.svg)](https://arxiv.org/abs/2510.02801) | **A $z\simeq0.4$ Little Red Dot analog: An Extended Starburst with an Overmassive Black hole**  |
|| X. Chen, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-10-06*|
|*Comments*| *18 pages, 9 figures, submitted to ApJ*|
|**Abstract**|            One of the most remarkable discoveries of JWST is a population of compact, red sources at z > 4, commonly referred to as Little Red Dots (LRDs). Spectroscopic identifications reported that most LRDs are active galactic nuclei (AGNs), which are preferentially found around z~6 and could imply a key phase in the formation and growth of black holes (BHs) in the early universe. Photometric surveys at lower redshift have recently been carried out to trace their evolution across cosmic time, and a small number of LRDs have been spectroscopically identified at both Cosmic Noon and in the local universe. Here we report the discovery of one of the lowest-z analogs of LRDs, J204837.26-002437.2 (hereafter J2048) at z = 0.4332, using new Gemini-N/GMOS IFU observations combined with archival multi-band photometric SED data. The GMOS data reveal extended blue emission from starburst with a star formation rate of 400 Msun yr-1, together with an extended, highly fast ionized outflow. This is the first spectroscopic confirmation of extended host emission and outflow in an LRD-like galaxy, providing a unique laboratory for understanding the nature of their high-redshift counterparts. Moreover, J2048 would host an extremely overmassive BH with a BH-to-stellar mass ratio of 0.6, with the BH mass and host stellar mass estimated to be 10^10.2 and 10^10.4 Msun, respectively. We discuss the origin and evolutionary fate of J2048, and the implications that such low-z analogs have for interpreting the properties of high-z LRDs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.03037/./GRG-II_Fig4A.png', 'tmp_2510.03037/./GRG-II_Fig12.png', 'tmp_2510.03037/./GRG-II_Fig2.png']
copying  tmp_2510.03037/./GRG-II_Fig4A.png to _build/html/
copying  tmp_2510.03037/./GRG-II_Fig12.png to _build/html/
copying  tmp_2510.03037/./GRG-II_Fig2.png to _build/html/
exported in  _build/html/2510.03037.md
    + _build/html/tmp_2510.03037/./GRG-II_Fig4A.png
    + _build/html/tmp_2510.03037/./GRG-II_Fig12.png
    + _build/html/tmp_2510.03037/./GRG-II_Fig2.png
found figures ['tmp_2510.03221/./sky_plot.png', 'tmp_2510.03221/./complet.png', 'tmp_2510.03221/./sky_model.png']
copying  tmp_2510.03221/./sky_plot.png to _build/html/
copying  tmp_2510.03221/./complet.png to _build/html/
copying  tmp_2510.03221/./sky_model.png to _build/html/
exported in  _build/html/2510.03221.md
    + _build/html/tmp_2510.03221/./sky_plot.png
    + _build/html/tmp_2510.03221/./complet.png
    + _build/html/tmp_2510.03221/./sky_model.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cf}{\color{magenta}}$
$\newcommand{\fpath}{figures/}$
$\newcommand{\labelenumi}{\Roman{enumi}.}$</div>



<div id="title">

# Probing the formation of megaparsec-scale giant radio galaxies\ II. Continuum \& polarization behavior from MHD simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.03037-b31b1b.svg)](https://arxiv.org/abs/2510.03037)<mark>Appeared on: 2025-10-06</mark> -  _Accepted for publication in A&A journal_

</div>
<div id="authors">

G. Giri, et al. -- incl., <mark>C. Fendt</mark>

</div>
<div id="abstract">

**Abstract:** The persistence of radiative signatures in giant radio galaxies (GRGs $\gtrsim 700$ kpc) remains a frontier topic of research, with contemporary telescopes revealing intricate features that require investigation. This study aims to examine the emission characteristics of simulated GRGs, and correlate them with their underlying three-dimensional dynamical properties. Sky-projected continuum and polarization maps at 1 GHz were computed from five 3D relativistic magnetohydrodynamical (RMHD) simulations by integrating the synthesized emissivity data along the line of sight, with the integration path chosen to reflect the GRG evolution in the sky plane. The emissivities were derived from these RMHD simulations, featuring FR-I and FR-II jets injected from different locations of the large-scale environment, with propagation along varying jet frustration paths. Morphologies, such as widened lobes from low-power jets and collimated flows from high-power jets, are strongly shaped by the triaxiality of the environment, resulting in features like wings and asymmetric cocoons, thereby making morphology a crucial indicator of GRG formation mechanisms. The decollimation of the bulk flow in GRG jets gives rise to intricate cocoon features, most notably filamentary structures—magnetically dominated threads with lifespans of a few Myr. High-jet-power cases frequently display enhanced emission zones at mid-cocoon distances (alongside warmspots around the jet-head), contradicting the interpretations of the GRG as a restarting source. In such cases, examining the lateral intensity variation of the cocoon may reveal the source's state, with a gradual decrease in emission suggesting a low-active stage. This study highlights that applying a simple radio power–jet power relation to a statistical GRG sample is unfeasible, as it depends on growth conditions of individual GRGs. Effects such as inverse-Compton CMB cooling and matter entrainment significantly impact the long-term emission persistence of GRGs. The diminishing fractional polarization with GRG evolution reflects increasing turbulence, underscoring the importance of modeling this characteristic further, particularly for even larger-scaled sources.

</div>

<div id="div_fig1">

<img src="tmp_2510.03037/./GRG-II_Fig4A.png" alt="Fig2" width="100%"/>

**Figure 2. -** A. Intensity maps (log $I_{\nu}$) at 1 GHz radio frequency are presented for two simulation cases: a low-power jet ("GRG\_lp\_min"; _top_) and a high-power jet ("GRG\_hp\_min"; _bottom_), both propagating along the minor axis of the ambient medium. While the overall structures resemble the dynamical maps from \citetalias{Giri2025}, the emission maps offer deeper insights, highlighting crucial aspects of the observability of these simulated giant radio structures at their evolved stages ($\sim$ 167 Myr: _top_; $\sim$ 69 Myr: _bottom_). (*Fig:Emission_minor*)

</div>
<div id="div_fig2">

<img src="tmp_2510.03037/./GRG-II_Fig12.png" alt="Fig15" width="100%"/>

**Figure 15. -** Intensity variation maps (as background) for five simulated scenarios at their evolved stages, presented as colormaps, overlaid with projected magnetic field distributions. The line lengths indicate proportionality to the corresponding fractional polarization (FP) values at each point. Median FP values are noted for `GRG\_lp\_min' (41.9\%), `GRG\_hp\_min' (45.8\%), `GRG\_lp\_maj' (38.4\%), `GRG\_hp\_maj' (39.2\%), and 'GRG\_hp\_edge' (35.1\%), and are annotated in each subfigure alongside a scale bar for reference. (*Fig:Polarization*)

</div>
<div id="div_fig3">

<img src="tmp_2510.03037/./GRG-II_Fig2.png" alt="Fig10" width="100%"/>

**Figure 10. -** Sliced velocity maps ($x-y$ plane, $z=0$) for the resultant giant radio galaxy morphologies from five simulation cases (labels indicated in each plot) are shown, illustrating the underlying matter-transport mechanisms responsible for these structures. These maps are displayed at dynamical ages of $\sim$ 167 Myr (GRG\_lp\_min), 196 Myr (GRG\_lp\_maj), 49 Myr (GRG\_hp\_edge), 137 Myr (GRG\_hp\_maj), and 69 Myr (GRG\_hp\_min), as also highlighted in \citetalias{Giri2025}. The colormap represents the $x-$component of velocity, capturing the bulk-flow (in blue) and back-flow (in red) dynamics, while the contours depict the $y-$component of velocity, with solid lines indicating $0.005c$ and dashed lines indicating $-0.005c$, to highlight the behavior of secondary structures forming within the domain. To analyze the impact of jet power on morphology and the influence of the tri-axial environment on the resulting structure, comparisons are made column-wise and row-wise, respectively. (*Fig:Velocity_dynamics*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.03037"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ml}[1]{\textcolor{red}{#1}}$
$\newcommand{\cmj}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\bs}[1]{\boldsymbol{#1}}$
$\newcommand{\vec}[1]{\boldsymbol{#1}}$</div>



<div id="title">

# Inferring Stellar Densities with Flexible Models I: The Distribution of RR Lyrae in the Milky Way with _Gaia_ DR3

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.03221-b31b1b.svg)](https://arxiv.org/abs/2510.03221)<mark>Appeared on: 2025-10-06</mark> -  _13 pages, 6 figures, submitted to ApJ, comments welcome!_

</div>
<div id="authors">

M. Lucey, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** Understanding the formation and evolutionary history of the Milky Way requires detailed mapping of its stellar components, which preserve fossil records of the Galaxy's assembly through cosmic time. RR Lyrae stars are particularly well-suited for this endeavor, as they are old, standard candle variables that probe the Galaxy's earliest formation epochs. In this work, we employ a hierarchical Bayesian Gaussian Mixture Model (GMM) to characterize the three-dimensional density distribution of RR Lyrae stars in the Milky Way. This approach provides a flexible framework for modeling complex stellar distributions, particularly in the inner Galaxy where the bulge, disk, and halo components overlap. Our analysis reveals that the inner Galaxy is dominated by a distinct prolate stellar population with axis ratio $q$ =1.30. Consistent with previous work, we find the halo follows a $r^{-4}$ power-law profile that flattens within 12 kpc of the Galactic center. We also confirm the halo is oblate $q$ =0.62 with a tilt angle of $12.22^{\circ}$ . We report for the first time that this tilt aligns the halo major axis in the direction of the Sagittarius dwarf galaxy. These results establish GMMs as an effective and flexible tool for modeling Galactic structure and provide new constraints on the distribution of old stars in the inner Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2510.03221/./sky_plot.png" alt="Fig2" width="100%"/>

**Figure 2. -** The sky distribution in Galactic coordinates ($l,b$) of our RR Lyrae sample. stars in our sample. The top panel shows the log density for the entire sample of 128,353 RR Lyrae. After removing the XMCs with a cut in RA and DEC, there are 105,325 RRL that we use in our analysis as shown in the middle panel. The bottom panel shows the sky distribution of our final sample colored by the _Gaia_ G magnitude, which, because of their standard-candle nature, is strongly correlated with the distance.  (*fig:sky*)

</div>
<div id="div_fig2">

<img src="tmp_2510.03221/./complet.png" alt="Fig3" width="100%"/>

**Figure 3. -** Empirical Completeness Maps of the RR Lyrae sample in Galactic Coordinates at distances of 5 kpc (top left), 10 kpc (middle left), and 20 kpc (bottom left). We show corresponding slices of the data in distance bins of 1 kpc in the right panels. At small distances, the completeness is degraded because of the bright limits of surveys. The dominant structure in the completeness map is consistent with the _Gaia_ scanning law.   (*fig:sf*)

</div>
<div id="div_fig3">

<img src="tmp_2510.03221/./sky_model.png" alt="Fig4" width="100%"/>

**Figure 4. -**  Comparison of the sky projected densities of our model to the data in Galactic coordinates ($l, b$). The top left panel is sky projection of our model where the darkest patches indicate the highest density of RRL. The top right panel shows the same but convolved with our model for the selection function. The bottom panel shows the equivalent sky projection of the RRL sample used in this work. In general, the sky projected model is similar to the data when convolved with the selection.    (*fig:skymodel*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.03221"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

131  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

19  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
